In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode

tools = [TavilySearchResults(max_results=1)]

tool_executor = ToolNode(tools=tools)

## 모델 준비

1. 메시지와 함께 작동해야 합니다. 모든 에이전트의 상태를 메시지 형태로 표현할 것이므로, 메시지를 잘 처리할 수 있어야 합니다.
2. OpenAI 함수 호출과 함께 작동해야 합니다. 이는 OpenAI 모델이거나 유사한 인터페이스를 제공하는 모델이어야 함을 의미합니다.

해당 모델 필요사항은 LangGraph를 사용하기 위한 필요사항은 아님.

In [29]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [24]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(tool) for tool in tools]
model = model.bind_functions(functions)

## Agent State 정의
langgraph의 주요 그래프 타입은 StatefulGraph.<Br>
이 그래프는 각 노드로 전달되는 상태 객체에 의해 매개변수화. 각 노드는 해당 상태를 업데이트하기 위한 작업을 반환합니다.<br> 
이러한 작업은 상태에 대해 특정 속성을 SET(기존 값을 덮어쓰기)하거나 기존 속성에 ADD(추가)할 수 있습니다. <br>
SET 또는 ADD 여부는 그래프에서 구성할 수 있는 상태 객체를 주석 처리하여 표시됩니다.


In [25]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## 노드 정의

In [26]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# 계속할지 그만할지 결정하는 함수
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]

    if 'function_call' not in last_message.additional_kwargs:
        return 'end'
    else:
        return 'continue'

# 모델 호출 함수
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {'messages': [response]}

# execute tools 함수
def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]
    action = ToolInvocation(
        tool = last_message.additional_kwargs['function_call']['name'],
        tool_input = json.loads(last_message.additional_kwargs['function_call']['arguments']),
    )
    
    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content = str(response), name = action.tool)
    return {'messages': [function_message]}


## Graph 정의

In [27]:
from langgraph.graph import END, StateGraph

# 새로운 그래프 정의
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)


# 그래프 시작노드 설정
workflow.set_entry_point("agent")

# 조건부 엣지 설정
workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue': 'action',
        'end': END,
    },
)

# normal 엣지 추가
workflow.add_edge('action', 'agent')

# 그래프 컴파일
app = workflow.compile()


In [30]:
from langchain_core.messages import HumanMessage, SystemMessage
system_message = SystemMessage(content="You are a helpful assistant.")
inputs = {"messages": [system_message, 
                       HumanMessage(content="What is the weather in sf?")]}
app.invoke(inputs)

{'messages': [SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I\'m sorry, but I am unable to provide real-time weather updates. You can check the weather in San Francisco by using a weather website or app, or by searching "weather in San Francisco" on a search engine.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-c1a6cecb-8b5a-4c05-8fa9-e7193138aa4d-0')]}